In [1]:
%%writefile ../crypto_chaotic/audioClient.py
#!/usr/bin/env python

import pyaudio
import wave
from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 1
#WAVE_OUTPUT_FILENAME = "output.wav"

class EchoClientDatagramProtocol(DatagramProtocol):
    p = pyaudio.PyAudio()

    inputStream = p.open(format=FORMAT,
                         channels=CHANNELS,
                         rate=RATE,
                         input=True,
                         frames_per_buffer=CHUNK)

    #outputStream = p.open(format=FORMAT,
    #                      channels=CHANNELS,
    #                      rate=RATE,
    #                      output=True)
    framectr = 0
    
    def startProtocol(self):
        self.transport.connect('127.0.0.1', 8000)
        self.sendDatagram()
    
    def sendDatagram(self):
        print self.framectr
        if self.framectr <= 10: #int(RATE / CHUNK * RECORD_SECONDS):#if len(self.strings):
            data = ' ' #self.inputStream.read(CHUNK)
            self.transport.write(data)
        else:
            reactor.stop()
            self.inputStream.stop_stream()
            self.inputStream.close()
            #self.outputStream.stop_stream()
            #self.outputStream.close()
            self.p.terminate()
        self.framectr += 1

    def datagramReceived(self, datagram, host):
        #print 'Datagram received: ', repr(datagram)
        if self.outputStream:
            #self.outputStream.write(datagram)
            self.sendDatagram()

def main():
    protocol = EchoClientDatagramProtocol()
    t = reactor.listenUDP(0, protocol)
    reactor.run()

if __name__ == '__main__':
    main()

Overwriting ../crypto_chaotic/audioClient.py


In [ ]:
%%bash
pushd ../crypto_chaotic
python2 -m audioClient